In [ ]:
import sys
sys.path.append('../fuzzy-tools') # or just install the module
sys.path.append('../astro-lightcurves-handler') # or just install the module

In [ ]:
%load_ext autoreload
%autoreload 2
from lcclassifier.results.utils import get_model_names

lcset_name = 'test'
rootdir = 'save'
method = 'spm-mcmc-estw'
cfilename = f'survey=alerceZTFv7.1~bands=gr~mode=onlySNe~method={method}'
kf = '1'

model_names = get_model_names(rootdir, cfilename, kf, lcset_name)
model_names

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import fuzzytools.files as ftfiles
from fuzzytools.datascience import misclassifications
import matplotlib.pyplot as plt

model_name = 'mdl=ParallelTimeModAttn~input_dims=1~dummy_seft=1~m=24~kernel_size=1~heads=4~time_noise_window=6*24**-1~enc_emb=g64-g64.r64-r64~dec_emb=g1-g128.r1-r128~b=203~pb=.~bypass_synth=0~bypass_prob=0.0~ds_prob=0.1'
rootdir = 'save/paper_v3'
train_mode = 'fine-tuning'
load_roodir = f'{rootdir}/{model_name}/{train_mode}/performance/{cfilename}'
print(load_roodir)
files, files_ids, kfs = ftfiles.gather_files_by_kfold(load_roodir, kf, lcset_name,
    fext='d',
    imbalanced_kf_mode='ignore', # error oversampling
    )
print(f'{files_ids}({len(files_ids)}#); {model_name}')

file_idx = 0
file = files[file_idx]
thdays = file()['thdays']
thday = thdays[-1]
thdays_predictions = file()['thdays_predictions'][thday]
y_pred_p = thdays_predictions['y_pred_p']
y_true = thdays_predictions['y_true']
class_names = file()['class_names']
obj_ids = file()['lcobj_names']

fig, axs, miss_objs_df = misclassifications.plot_misclassification_map(y_pred_p, y_true, class_names,
    obj_ids=obj_ids,
    #pred_prob_th=.8, # None .5
    fontsize=12,
    figsize=(20,18),
    legend_loc='upper right',
    #verbose=1,
    also_show_correct_objs_txt=True,
    )
display(miss_objs_df)
plt.show()

In [ ]:
txt = ''
for miss_obj_id in miss_objs_df.index:
    txt += f"'{miss_obj_id}', "
print(f'deep_miss_obj_ids = [{txt}]')